<a href="https://colab.research.google.com/github/WangKehanK/CS505/blob/main/Twitter_User_Age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import json
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Twitter_user_handles_to_predict.csv", encoding = "utf-8")
df_labeled = pd.read_csv("/content/drive/My Drive/Twitter_users_labeled_with_age_and_gender.csv", encoding = "latin-1")

In [ ]:
# Get authentication for tweepy from auth.txt
# auth.txt was uploaded before in order to use m3inference
keys = ''
with open(f'/content/drive/MyDrive/auth.txt', 'r') as reader:
  keys=reader.read()
keys = keys.split('\n')
# for i in range(len(keys)):
#   keys[i] = keys[i].split('"')[1];


In [ ]:
# Setup tweepy
import tweepy
api_key = keys[0]
api_secret = keys[1]
access_token = keys[2]
access_secret = keys[3]
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
# Try first user
user = df['Username']
userID = user[0]
print(userID, len(user))
tweets = api.user_timeline(screen_name=userID,count=100,include_rts = False,tweet_mode = 'extended')

In [ ]:
# Extract full text from tweets
# for testing purpose, print user's 3 most recent tweets

tweet_text = [tweet._json['full_text'] for tweet in tweets]
for info in tweets[:3]:
    print("ID: {}".format(info.id))
    print(info.created_at)
    print(info.full_text)
    print("\n")
print(len(tweet_text))

In [ ]:
# Extract latest 100 tweets using api.user_timeline for each user
import time

all_tweets = []
for sleep_count, userID in enumerate(user):
  try:
    tweets = api.user_timeline(screen_name=userID, 
                              count=100,
                              include_rts = False,
                              tweet_mode = 'extended')
    all_tweets.extend(tweets)
    print('Number of tweets extracted {}'.format(len(all_tweets)))

    if (sleep_count %450 == 0 and not sleep_count == 0):
      time.sleep(15*60)
      print("Taking a sleep break..." + '\n')
      
  except tweepy.TweepError as e:
    # print(e.response.status_code, e.reason)
    # the user not found error
    print("screen_name that failed=",  userID)
    pass

print("Extraction Done")

In [ ]:
outtweets = [[tweet.id_str, 
              tweet.created_at, 
              tweet.favorite_count, 
              tweet.retweet_count, 
              tweet.full_text.encode("utf-8").decode("utf-8")] 
             for idx,tweet in enumerate(all_tweets)]

df = DataFrame(outtweets,columns=["id","created_at","favorite_count","retweet_count", "text"])
df.to_csv('%all_tweets.csv' % userID,index=False)
df.head(3)


TODO: Using BERT to extract fixed feature vectors

https://github.com/google-research/bert#using-bert-to-extract-fixed-feature-vectors-like-elmo

In [ ]:
age_no_na = df_labeled[df_labeled['human.labeled.age'].notna()]
age_no_na